In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append('../')    
import GSSUtility as GU
import statsmodels.formula.api as smf
from pandas.rpy import common as com

/home/misha/python/pandas-0.16.0-py2.7-linux-x86_64.egg/pandas/rpy/__init__.py:8: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2, found here: http://rpy.sourceforge.net
  "like rpy2, found here: http://rpy.sourceforge.net", FutureWarning)


In [2]:
%rm ../GSSUtility.pyc # remove this file because otherwise it will be used instead of the updated .py file
reload(GU)

<module 'GSSUtility' from '../GSSUtility.py'>

In [3]:
pathToData = '../../Data/'
dataCont = GU.dataContainer(pathToData)

Loading DataFrame df. This may take a few minutes.


In [3]:
def independent_columns(A, tol = 1e-05):
    """
    Return an array composed of independent columns of A.

    Note the answer may not be unique; this function returns one of many
    possible answers.

    http://stackoverflow.com/q/13312498/190597 (user1812712)
    http://math.stackexchange.com/a/199132/1140 (Gerry Myerson)
    http://mail.scipy.org/pipermail/numpy-discussion/2008-November/038705.html
        (Anne Archibald)

    >>> A = np.array([(2,4,1,3),(-1,-2,1,0),(0,0,2,2),(3,6,2,5)])
    >>> independent_columns(A)
    np.array([[1, 4],
              [2, 5],
              [3, 6]])
    """
    Q, R = np.linalg.qr(A.dropna())
    independent = np.where(np.abs(R.diagonal()) > tol)[0]
    print independent
    return A.iloc[:, independent]

In [3]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
r = robjects.r
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import pandas.rpy.common as com
# import GSSUtility as GU
from rpy2.robjects.packages import importr
# R's "base" package
amelia = importr('Amelia')
mi = importr('mi')
mi = r['mi']
df = pd.read_csv('../../Data/test_collinear.csv', index_col=0)
# df.index = range(len(df))

In [36]:
articlesToUse = GU.filterArticles(dataCont.articleClasses, GSSYearsUsed=True, GSSYearsPossible=False, centralIVs=True)
article = [a for a in articlesToUse if a.articleID == 7449][0]

In [37]:
import random
print [a.articleID for a in random.sample(articlesToUse, 20)]

[2596, 7432, 7700, 3567, 1611, 5210, 5448, 5015, 1686, 1946, 6524, 2581, 638, 2822, 6156, 633, 9, 2243, 5374, 175]


In [38]:
article.IVs, article.DVs, article.centralIVs, article.GSSYearsUsed

(['DENOM16', 'OTH16', 'RELIG16'],
 ['DENOM', 'FUND', 'RELIG'],
 ['DENOM16', 'OTH16', 'RELIG16'],
 [1973,
  1974,
  1975,
  1976,
  1977,
  1978,
  1980,
  1982,
  1983,
  1984,
  1985,
  1986,
  1987,
  1988,
  1989,
  1990])

In [14]:
formula2 = '''
standardize(FUND, ddof=1) ~ C(RELIG)'''
DV = article.DVs[0]

In [42]:
design = dataCont.df.loc[1973, [DV] + article.IVs]
design.index = range(len(design))
design.head()

,FUND,DENOM16,OTH16,RELIG16
0,1,38,NaN,1
1,2,38,NaN,1
2,2,38,NaN,1
3,2,NaN,NaN,4
4,2,38,NaN,1


In [17]:
# mi(com.convert_to_r_dataframe(design))

In [30]:
# summary = r['summary']

In [18]:
# print r('summary(imp$imp)')

In [19]:
# print com.convert_robj(r['imp1']).DEGREE.value_counts()
# print
# print design.DEGREE.value_counts()

In [20]:
# rcode='''
#     library(mi)
#     mydf = %s
#     IMP = mi(mydf, n.imp=2, n.iter=6, max.minutes=1)
#     imp1 <- mi.data.frame(IMP, m = 1)
# ''' % com.convert_to_r_dataframe(design).r_repr()
# r(rcode)
# com.convert_robj(r['imp1'])

In [45]:
res=GU.runModel(dataCont, 1973, DV, article.IVs)

IV OTH16 is of type "DONOTUSE"


In [46]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     standardize(FUND, ddof=1)   R-squared:                       0.009
Model:                                   OLS   Adj. R-squared:                 -0.006
Method:                        Least Squares   F-statistic:                    0.5915
Date:                       Tue, 21 Apr 2015   Prob (F-statistic):              0.555
Time:                               14:56:06   Log-Likelihood:                -198.97
No. Observations:                        141   AIC:                             403.9
Df Residuals:                            138   BIC:                             412.8
Df Model:                                  2                                         
Covariance Type:                   nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept             -0.8197      1.003     -0.817      0.415        -2.803     1.163
C(DENOM16)[T.60.0]     0.8183      1.007      0.813      0.418        -1.172     2.809
C(DENOM16)[T.70.0]     1.3285      1.228      1.082      0.281        -1.100     3.757
==============================================================================
Omnibus:                       21.894   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.887
Skew:                           0.773   Prob(JB):                     0.000131
Kurtosis:                       2.193   Cond. No.                         29.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
for year in article.GSSYearsUsed:
    design = dataCont.df.loc[year, [DV] + article.IVs]
#     design = design.fillna(design.mean())
    formula = GU.createFormula(dataCont, design)
#     results = smf.ols(formula2, data=design.dropna()).fit()

    if len(design.dropna()) <= design.shape[1]: 
        nominals = GU.createFormula(dataCont, design, return_nominals=True)
        non_nominals = list(set(design.columns) - set(nominals)) # list because sets are unhashable and cant be used for indices
        if len(non_nominals)>0: 
            design[non_nominals] = design[non_nominals].fillna(design[non_nominals].mean()) # the naive way
        if len(nominals)>0:
            design[nominals] = design[nominals].fillna(design[nominals].mode())
    
    design = GU.removeConstantColumns(design.dropna())
    print smf.ols(formula, data=design.dropna()).fit().summary()
    break

IV OTH16 is of type "DONOTUSE"
Dropping column RELIG16 because it is constant


NameError: name 'RELIG16' is not defined

In [49]:
print formula
design.head()

standardize(FUND, ddof=1) ~ C(DENOM16) + C(RELIG16)


,FUND,DENOM16,OTH16
year,,,
1973,3,60,81
1973,2,60,40
1973,2,60,23
1973,2,60,46
1973,2,60,51


In [68]:
DV = 'GRASS'
for year in article.GSSYearsUsed:
    design = dataCont.df.loc[year, [DV] + article.IVs]
    design = design.fillna(design.mean())
    formula = GU.createFormula(dataCont, design)
#     results = smf.ols(formula2, data=design.dropna()).fit()
    
           
    print smf.ols(formula, data=design.dropna()).fit().summary()
# 

                                OLS Regression Results                                
Dep. Variable:     standardize(GRASS, ddof=1)   R-squared:                       0.101
Model:                                    OLS   Adj. R-squared:                  0.099
Method:                         Least Squares   F-statistic:                     42.02
Date:                        Thu, 09 Apr 2015   Prob (F-statistic):           1.98e-33
Time:                                15:55:15   Log-Likelihood:                -2046.6
No. Observations:                        1499   AIC:                             4103.
Df Residuals:                            1494   BIC:                             4130.
Df Model:                                   4                                         
Covariance Type:                    nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------

In [76]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../Data/test_collinear.csv', index_col=0)
from scipy.stats import pearsonr

In [5]:
df.head()

,HAPMAR,AGE,AGEWED,ATTEND,CHILDS,DIVORCE,EARNRS,EDUC,HOMPOP,INCOME,MARITAL,PRESTIGE,RACE,RINCOME,SEX,SPEDUC,SPWRKSTA,VOTE72,WRKSTAT,XNORCSIZ
0,1,56,23,7,1,2,2,12,2,NaN,1,46,1,NaN,2,12,1,1,2,3
1,2,51,20,1,1,2,1,12,2,12,1,45,1,NaN,2,12,1,2,7,3
2,2,41,23,3,3,2,3,16,5,12,1,69,1,12,1,12,2,1,1,3
3,1,55,21,1,2,2,2,12,5,10,1,44,1,9,1,12,7,1,1,3
4,1,32,24,3,3,2,1,18,5,11,1,60,1,11,1,12,7,1,1,3


#Imputing with GLRM

In [4]:
from glrm import GLRM
from glrm.loss import QuadraticLoss, HingeLoss
from glrm.reg import QuadraticReg
from glrm.convergence import Convergence

In [80]:
GLRM?

In [5]:
regX, regY = QuadraticReg(0.01), QuadraticReg(0.01)

converge = Convergence(TOL = 1e-5, max_iters = 100)

model = GLRM(df.values, QuadraticLoss, regX, regY, k=2, converge=converge)
model.fit()

X, Y = model.factors()
A_hat = model.predict() # a horizontally concatenated matrix, not a list

norm(A_hat - hstack(A_list)) # by hand

SolverError: Solver 'ECOS' failed. Try another solver.